In [ ]:
# Backbone and Sunny
model_s = get_model_instance_segmentation(2)

# ROI_rain
model_r = get_model_instance_segmentation(2)
r = model_r.roi_heads
new_roi_heads = RoIHeadsRain(r.box_roi_pool, r.box_head, r.box_predictor, 
              r.proposal_matcher.high_threshold, r.proposal_matcher.low_threshold, 
              r.fg_bg_sampler.batch_size_per_image, r.fg_bg_sampler.positive_fraction,
              r.box_coder.weights, r.score_thresh, r.nms_thresh, r.detections_per_img,
              r.mask_roi_pool, r.mask_head, r.mask_predictor,
              r.keypoint_roi_pool, r.keypoint_head, r.keypoint_predictor)
# RPN_rain
rr = model_r.rpn
pre_nms_top_n = dict(training=2000, testing=1000)
post_nms_top_n = dict(training=2000, testing=1000)
new_rpn = RegionProposalNetworkRain(rr.anchor_generator, rr.head, fg_iou_thresh = 0.7, bg_iou_thresh = 0.3,
                                     batch_size_per_image = 256, positive_fraction = 0.5, pre_nms_top_n = pre_nms_top_n, 
                                     post_nms_top_n = post_nms_top_n, nms_thresh = 0.7, score_thresh = 0.0)
model_r.roi_heads = new_roi_heads
model_r.rpn = new_rpn

#######

# ROI_fog
model_f = get_model_instance_segmentation(2)
f = model_f.roi_heads
f_new_roi_heads = RoIHeadsFog(f.box_roi_pool, f.box_head, f.box_predictor, 
              f.proposal_matcher.high_threshold, f.proposal_matcher.low_threshold, 
              f.fg_bg_sampler.batch_size_per_image, f.fg_bg_sampler.positive_fraction,
              f.box_coder.weights, f.score_thresh, f.nms_thresh, f.detections_per_img,
              f.mask_roi_pool, f.mask_head, f.mask_predictor,
              f.keypoint_roi_pool, f.keypoint_head, f.keypoint_predictor)
# RPN_fog
ff = model_f.rpn
f_new_rpn = RegionProposalNetworkFog(ff.anchor_generator, ff.head, fg_iou_thresh = 0.7, bg_iou_thresh = 0.3,
                                     batch_size_per_image = 256, positive_fraction = 0.5, pre_nms_top_n = pre_nms_top_n, 
                                     post_nms_top_n = post_nms_top_n, nms_thresh = 0.7, score_thresh = 0.0)
model_f.roi_heads = f_new_roi_heads
model_f.rpn = f_new_rpn

########

# ROI_night
model_ni = get_model_instance_segmentation(2)
ni = model_ni.roi_heads
ni_new_roi_heads = RoIHeadsNight(ni.box_roi_pool, ni.box_head, ni.box_predictor, 
              ni.proposal_matcher.high_threshold, ni.proposal_matcher.low_threshold, 
              ni.fg_bg_sampler.batch_size_per_image, ni.fg_bg_sampler.positive_fraction,
              ni.box_coder.weights, ni.score_thresh, ni.nms_thresh, ni.detections_per_img,
              ni.mask_roi_pool, ni.mask_head, ni.mask_predictor,
              ni.keypoint_roi_pool, ni.keypoint_head, ni.keypoint_predictor)
# RPN_night
nini = model_ni.rpn
ni_new_rpn = RegionProposalNetworkNight(nini.anchor_generator, nini.head, fg_iou_thresh = 0.7, bg_iou_thresh = 0.3,
                                     batch_size_per_image = 256, positive_fraction = 0.5, pre_nms_top_n = pre_nms_top_n, 
                                     post_nms_top_n = post_nms_top_n, nms_thresh = 0.7, score_thresh = 0.0)
model_ni.roi_heads = ni_new_roi_heads
model_ni.rpn = ni_new_rpn

In [ ]:
class NewModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.transform = model_s.transform
        self.backbone = model_s.backbone
        self.rpn_sunny = model_s.rpn
        self.rpn_rain = model_r.rpn
        self.rpn_fog = model_f.rpn
        self.rpn_night = model_ni.rpn
        self.roi_sunny = model_s.roi_heads
        self.roi_rain = model_r.roi_heads     
        self.roi_fog = model_f.roi_heads
        self.roi_night = model_ni.roi_heads

    def eager_outputs_sunny(self, losses_sunny, detections_sunny):
        if self.training:
            return losses_sunny    
        return detections_sunny    
    def eager_outputs_rain(self, losses_rain, detections_rain):
        if self.training:
            return losses_rain      
        return detections_rain    
    def eager_outputs_fog(self, losses_fog, detections_fog):
        if self.training:
            return losses_fog      
        return detections_fog   
    def eager_outputs_night(self, losses_night, detections_night):
        if self.training:
            return losses_night    
        return detections_night
        
    def forward(self, weather, images, targets=None):
        if self.training:
            if targets is None:
                torch._assert(False, "targets should not be none when in training mode")
            else:
                for target in targets:
                    boxes = target["boxes"]
                    if isinstance(boxes, torch.Tensor):
                        torch._assert(
                            len(boxes.shape) == 2 and boxes.shape[-1] == 4,
                            f"Expected target boxes to be a tensor of shape [N, 4], got {boxes.shape}.",
                        )
                    else:
                        torch._assert(False, f"Expected target boxes to be of type Tensor, got {type(boxes)}.")

        original_image_sizes: List[Tuple[int, int]] = []
        for img in images:
            val = img.shape[-2:]
            torch._assert(
                len(val) == 2,
                f"expecting the last two dimensions of the Tensor to be H and W instead got {img.shape[-2:]}",
            )
            original_image_sizes.append((val[0], val[1]))

        images, targets = self.transform(images, targets)
        # Check for degenerate boxes
        # TODO: Move this to a function
        if targets is not None:
            for target_idx, target in enumerate(targets):
                boxes = target["boxes"]
                degenerate_boxes = boxes[:, 2:] <= boxes[:, :2]
                if degenerate_boxes.any():
                    # print the first degenerate box
                    bb_idx = torch.where(degenerate_boxes.any(dim=1))[0][0]
                    degen_bb: List[float] = boxes[bb_idx].tolist()
                    torch._assert(
                        False,
                        "All bounding boxes should have positive height and width."
                        f" Found invalid box {degen_bb} for target at index {target_idx}.",
                    )
        #backbone
        features = self.backbone(images.tensors)
        if isinstance(features, torch.Tensor):
            features = OrderedDict([("0", features)]) 
        
        ############################MODIFIED#######################
        # Classify
        if weather[0] == 'fog':
            features_fog = features
            #rpn
            proposals_fog, proposal_losses_fog = self.rpn_fog(images, features_fog, targets)         
            #roi_heads
            detections_fog, detector_losses_fog = self.roi_fog(features_fog, proposals_fog, images.image_sizes, targets)
            detections_fog = self.transform.postprocess(detections_fog, images.image_sizes, original_image_sizes)  # type: ignore[operator]

            losses_fog = {}
            losses_fog.update(detector_losses_fog)
            losses_fog.update(proposal_losses_fog)

            if torch.jit.is_scripting():
                if not self._has_warned:
                    warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
                    self._has_warned = True
                return losses_fog, detections_fog
            else:
                return self.eager_outputs_fog(losses_fog, detections_fog)
            
        elif weather[0] == 'rain':
            features_rain = features
            #rpn
            proposals_rain, proposal_losses_rain = self.rpn_rain(images, features_rain, targets)
        
            #roi_heads
            detections_rain, detector_losses_rain = self.roi_rain(features_rain, proposals_rain, images.image_sizes, targets)
            detections_rain = self.transform.postprocess(detections_rain, images.image_sizes, original_image_sizes)  # type: ignore[operator]

            losses_rain = {}
            losses_rain.update(detector_losses_rain)
            losses_rain.update(proposal_losses_rain)

            if torch.jit.is_scripting():
                if not self._has_warned:
                    warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
                    self._has_warned = True
                return losses_rain, detections_rain
            else:
                return self.eager_outputs_rain(losses_rain, detections_rain)

        elif weather[0] == 'sunny':
            features_sunny = features
            #rpn
            proposals_sunny, proposal_losses_sunny = self.rpn_sunny(images, features_sunny, targets)
        
            #roi_heads
            detections_sunny, detector_losses_sunny = self.roi_sunny(features_sunny, proposals_sunny, images.image_sizes, targets)
            detections_sunny = self.transform.postprocess(detections_sunny, images.image_sizes, original_image_sizes)  # type: ignore[operator]

            losses_sunny = {}
            losses_sunny.update(detector_losses_sunny)
            losses_sunny.update(proposal_losses_sunny)

            if torch.jit.is_scripting():
                if not self._has_warned:
                    warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
                    self._has_warned = True
                return losses_sunny, detections_sunny
            else:
                return self.eager_outputs_sunny(losses_sunny, detections_sunny)
            
        elif weather[0] == 'night':
            features_night = features
            #rpn
            proposals_night, proposal_losses_night = self.rpn_night(images, features_night, targets)
        
            #roi_heads
            detections_night, detector_losses_night = self.roi_night(features_night, proposals_night, images.image_sizes, targets)
            detections_night = self.transform.postprocess(detections_night, images.image_sizes, original_image_sizes)  # type: ignore[operator]

            losses_night = {}
            losses_night.update(detector_losses_night)
            losses_night.update(proposal_losses_night)

            if torch.jit.is_scripting():
                if not self._has_warned:
                    warnings.warn("RCNN always returns a (Losses, Detections) tuple in scripting")
                    self._has_warned = True
                return losses_night, detections_night
            else:
                return self.eager_outputs_night(losses_night, detections_night)
            
model_ex2 = NewModel()